# DATA MINING PROJECT: Analysis of a Supermarket’s Customers
## 4) Pattern Mining
### *Antonio Strippoli, Valerio Mariani*

In [ ]:
from gsp import apriori
import pandas as pd
import datetime
import logging
import pickle
import time
import os

# Set logging
if os.path.exists('log.txt'):
    os.remove('log.txt')
logging.basicConfig(level=logging.INFO, filename="log.txt", filemode="a+", format="%(message)s")
logging.getLogger().addHandler(logging.StreamHandler())

### Functions

In [ ]:
def read_dataset():
    """Read the dataset using Pandas, keeping only bought items."""
    df = pd.read_csv("../DM_25_TASK1/customer_supermarket_2.csv", index_col=0, parse_dates=["PurchaseDate"])
    return df[df['Qta'] > 0]

In [ ]:
def remove_baskets(df, threshold):
    """Keep only customers with more than `threshold` baskets."""
    customers = df.groupby('CustomerID').agg({'BasketID': 'nunique'})
    customers = customers[customers >= threshold].dropna().index.values
    return df[df['CustomerID'].isin(customers)]

In [ ]:
def sequentialize(df, return_times=False):
    """Convert a dataset into its sequential form. It can also return the time stamps of the baskets."""
    seq_data = []
    times = []
    for customer in df.groupby('CustomerID'):
        customer = customer[1]
        tmp = []
        tmp2 = []
        for basket in customer.groupby('BasketID'):
            basket = basket[1]
            purchases = list( basket['ProdID'].unique() )
            time = basket['PurchaseDate'].max()
            tmp.append(purchases)
            tmp2.append(time)
        seq_data.append(tmp)
        times.append(tmp2)
    if not return_times:
        return seq_data
    return seq_data, times

In [ ]:
def save_to_pickle(result_set, min_baskets, min_sup):
    """Save gsp results"""
    with open(f'gsp_res/{min_baskets}mb_{int(min_sup*100)}ms.pickle', 'wb') as handle:
        pickle.dump(result_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Apply GSP on sequential data

In [ ]:
# Main cycle: apply GSP multiple times
params = {
    'min_sup': [0.4, 0.35, 0.3, 0.25, 0.2, 0.15],
    'min_baskets': [20, 10, 5, 3, 2],
}
for min_sup in params['min_sup']:
    for min_baskets in params['min_baskets']:
        logging.info(f"MIN_BASKETS: {min_baskets}, MIN_SUP: {min_sup}")

        # Read the dataset
        df = read_dataset()
        # Remove some baskets
        df = remove_baskets(df, min_baskets)
        # Convert into seq form
        seq_data = sequentialize(df)
        
        # Apply GSP
        t0 = time.time()
        result_set = apriori(seq_data, min_sup, verbose=False)
        t1 = time.time()

        # Compute n. of sequences with len > 2 and n. of sequences containing duplicates
        cnt_len_2 = 0
        cnt_duplicates = 0
        for r in result_set:
            r = r[0]
            tmp = []
            for l in r:
                tmp.extend(l)
            if len(tmp) >= 2:
                cnt_len_2 += 1
                if len(set(tmp)) < len(tmp):
                    cnt_duplicates += 1

        logging.info(
            f"TOTAL TIME:\t{round(t1-t0, 2)} s\n"\
            f"LEN RESULT SET:\t{len(result_set)}\n"\
            f"LEN SEQ > 2:\t{cnt_len_2}\nN. DUPLICATES:\t{cnt_duplicates}\n"
        )

        # Save
        save_to_pickle(result_set, min_baskets, min_sup)

### Considerazioni

Prendendo quelli che hanno fatto almeno 20 baskets, otteniamo una mole maggiore di risultati. Abbassando min_baskets, il supporto comincia a diventare sempre più basso in generale, il che lascia comunque presagire che nei clienti più occasionali non ci siano pattern evidenti.

Il parametro min_baskets è importante perché altrimenti trovare dei pattern un po' più interessanti (che spazino tra basket diversi) è molto difficile (richiedono min_support bassi che alzano il costo computazionale). Alla fine abbiamo scelto 10 come giusto compromesso tra un numero di clienti abbastanza alto (il 10% di quelli di partenza) e sequenze variegate/lunghe.

### Analyze results and collect statistics

In [ ]:
# Config (which result do we want to analyze)
min_baskets = 10
min_sup = 25

# Read gsp results
with open(f'gsp_res/{min_baskets}mb_{min_sup}ms.pickle', 'rb') as handle:
    result_set = pickle.load(handle)
# Sort by support
result_set.sort(key=lambda x: x[1], reverse=True)
# Prepare a copy
result_set_original = result_set

# Read and prepare the dataset
df = read_dataset()
df = remove_baskets(df, min_baskets)

In [ ]:
# Compute distribution of the lengths of sequences and n. of sequences containing duplicates
cnt_len = {1:0, 2:0, 3:0, 4:0, 5:0}
cnt_duplicates = 0
for r in result_set:
    r = r[0]
    tmp = []
    for l in r:
        tmp.extend(l)
    len_tmp = len(tmp)
    cnt_len[len_tmp] += 1
    if len(set(tmp)) < len_tmp:
        cnt_duplicates += 1

print(f"Distribution of lengths: {cnt_len}")
print(f"Sequences containing duplicates: {cnt_duplicates} / {len(result_set)}")

In [ ]:
""" PSEUDOCODICE
per ogni cliente in clienti:
    per ogni risultato in risultati:
        per ogni carrello_1 in risultato: # carrello_res
            per ogni carrello_2 in cliente: # carrello_customer
                carrello_1 è contenuto in carrello_2?
                    SI: scorri carrello_1 col prossimo carrello (fino a quando non finiscono, se finiscono allora questo cliente è ok)
                    NO: scorri carrello_2 col prossimo carrello (fino a quando non finiscono, se finiscono allora questo cliente NON è ok)
"""
from copy import deepcopy

result_set = deepcopy(result_set_original)

# Prepare result_set
for i in range(len(result_set)):
    # Convert from tuple to list
    result_set[i] = list(result_set[i])
    tmp = []
    # Create a nested list for future storage of mean qta of each item in the patterns
    for basket in result_set[i][0]:
        tmp2 = []
        for item in basket:
            tmp2.append(0)
        tmp.append(tmp2)
    result_set[i].append(tmp)

# Find original transactions for each pattern and collect statistics
out = []
n_customers = len(df['CustomerID'].unique())
for customer_i, customer in enumerate(df.groupby('CustomerID')):
    # Progress
    print(f"{customer_i+1} / {n_customers}")
    # Extract baskets from the customer
    baskets_customer = list(enumerate([x[1] for x in customer[1].groupby('BasketID')]))
    
    for result_i in range(len(result_set)):
        res = result_set[result_i][0]

        # Compare the baskets in the result against those of the customer
        bc_i = 0
        transactions = []
        for basket_res in res:
            for i, basket_customer in baskets_customer[bc_i:]:
                entries = basket_customer[basket_customer['ProdID'].isin(basket_res)]
                entries = entries.groupby('ProdID').aggregate({'Qta': 'sum'})
                if len(entries) >= len(basket_res):
                    bc_i = i + 1
                    transactions.append(entries)
                    break
            else: # We iterated over all the baskets of the customer without finding a match for basket_res
                break
        else:
            # Compute qta for each item in the pattern
            for i, basket in enumerate(transactions):
                for j in range(len(basket)):
                    item = basket.iloc[j]
                    result_set[result_i][2][i][j] += item['Qta']

            out.append(transactions)

# Compute mean of the qta previously found
for res in result_set:
    min_sup = res[1]
    sup = min_sup * n_customers
    for i in range(len(res[2])):
        for j in range(len(res[2][i])):
            res[2][i][j] = round(res[2][i][j] / sup)

In [ ]:
# Convert ProductIDs to readable descriptions
for r_i, result in enumerate(result_set):
    tmp = []
    for b_i, basket in enumerate(result_set[r_i][0]):
        tmp2 = []
        for p_i, p in enumerate(basket):
            tmp2.append(df[df['ProdID'] == p]['ProdDescr'].iloc[0])
        tmp.append(tmp2)
    result_set[r_i][0] = tmp

In [ ]:
result_set

### Considerazioni su RESULT_SET
- Conta sequenze che hanno doppioni e quelle che non ce l'hanno (in percentuale?)
- ogni evento è formato da un singolo elemento (non ci sono carrelli che hanno in comune più di un elemento, ma solo sequenze di carrelli con oggetti comuni)
- Contare quante sono sequenze lunghe 1, 2, 3...

### Considerazioni su transazioni e clienti a cui si riferiscono le sequenze
- quantità media di oggetti presa per ogni oggetto di ogni sequenza
- tempo passato tra una transazione e l'altra

### Considerazioni
due casi:
- oggetti uguali: l'oggetto vende bene? Vengono ricomprati per essere venduti ancora
- oggetti diversi: oggetto1 magari non ha venduto benissimo e se n'è comprato uno simile per provare a vendere quello, oppure ha comprato semplicemente un'altra variante.

### Apply Time Constraints

#### Sanity check

In [ ]:
# Read and prepare the dataset
df = read_dataset()
df = remove_baskets(df, 10)
seq_data, times = sequentialize(df, return_times=True)

print(times[0][0])
print(times[1][0])
print(type(times[0][0] - times[1][0]))

# for s, t in zip(seq_data, times):
#     if len(s) != len(t):
#         assert False

#### Re-apply the procedure introducing time constraints

In [ ]:
# Read the dataset
df = read_dataset()
# Remove some baskets
df = remove_baskets(df, 10)
# Convert into seq form
seq_data, times = sequentialize(df, return_times=True)

# Apply GSP
t0 = time.time()
result_set = apriori(seq_data, 0.25)
t1 = time.time()
print(t1-t0)

t0 = time.time()
result_set2 = apriori(seq_data, 0.25, time_stamps=times, min_gap=datetime.timedelta(days=1))
t1 = time.time()
print(t1-t0)

In [ ]:
print(len(result_set))
print(len(result_set2))